### Segment 읽어 오기 + 합치기

In [1]:
import json

stt_file_path = './samples/지식브런치_stt.json'
with open(stt_file_path, 'r') as f:
    stt_script = json.load(f)

segments = stt_script['segments']
print('Number of segments:', len(segments))

Number of segments: 50


In [2]:
stt_script

{'result': 'SUCCEEDED',
 'message': 'Succeeded',
 'token': '60fd7f10b0a1444ba5c0388d7e137864',
 'version': 'ncp_v2_v2.1.6-d90fef3-20230420__v4.1.5_ko_ncp_20221227_',
 'params': {'service': 'ncp',
  'domain': 'general',
  'lang': 'ko',
  'completion': 'sync',
  'callback': '',
  'diarization': {'enable': False,
   'speakerCountMin': -1,
   'speakerCountMax': -1},
  'boostings': [],
  'forbiddens': '',
  'wordAlignment': True,
  'fullText': True,
  'noiseFiltering': True,
  'priority': 0,
  'userdata': {'_ncp_DomainCode': 'basasak',
   '_ncp_DomainId': 5405,
   '_ncp_TaskId': 13470062,
   '_ncp_TraceId': 'ac127c49e71d441a9532e706c85a236c'}},
 'progress': 100,
 'keywords': {},
 'segments': [{'start': 160,
   'end': 12660,
   'text': '독일 제국의 철혈재상 비스마르크가 독일을 통일하고 보니 세계의 식민지는 이미 열강들의 차지였습니다. 단독 진출은 무리라고 판단한 비스마르크는 동양에서 협력자를 찾았습니다.',
   'confidence': 0.9481745,
   'diarization': {'label': ''},
   'speaker': {'label': '', 'name': '', 'edited': False},
   'words': [[470, 740, '독일'],
    [770, 1

In [3]:
from functools import reduce

# Segment 병합
text = ' '.join(map(lambda s: s['text'], segments))
words = list(reduce(lambda acc, cur: acc + cur['words'], segments, []))

In [4]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

splitted_sentences = list(map(lambda x: x.text, kiwi.split_into_sents(text)))

print('before split:', len(segments))
print('after naive split:', len(stt_script['text'].split('.')))
print('after kiwi split:', len(splitted_sentences))

before split: 50
after naive split: 85
after kiwi split: 90


### kiwi로 토큰화된 기준으로 words 재정렬

In [5]:
# 처리할 단어의 인덱스 (Queue의 역할을 수행하며, 증가하기만 한다.)
c_idx = 0

# 문장 앞뒤로 남아 있을 수 있는 whitespace를 전처리
splitted_sentences = list(map(str.strip, splitted_sentences))

# 스피치를 구성하는 단어를 앞에서부터 하나씩 빼서 문장을 재구성한다.
# 이때 kiwi와 STT 결과가 서로 단어를 다른 단위로 인식한다면(예: kiwi는 '안녕하세요'를 하나의 단어로 인식하고, STT는 '안녕'과 '하세요'로 인식한다면)
# STT의 결과를 우선적으로 사용한다.

# 재조합된 문장들을 담을 리스트 (element는 아래 `current`를 참고)
reconstructed_segments = []
for s_idx, sentence in enumerate(splitted_sentences):
    current = {
        'start': None,
        'end': None,
        'text': sentence[:],
        'words': [],
    }

    # 원본 문자열을 수정하지 않도록 하기 위해 사용
    processing_sentence = sentence[:]
    
    # 해당 문장이 STT 결과와 kiwi 결과가 달라 깨져 있는지 여부
    broken = False

    # 종료 조건: 문장이 빈 문자열이 되면 종료
    while True:
        processing_sentence = processing_sentence.lstrip()
        # 단 문장의 맨 앞에 현재 처리해야 할 단어가 있는지 확인하므로, 앞에 포함된 공백을 삭제함
        if not processing_sentence:
            break

        # 현재 처리해야 할 단어에 대해,
        start, end, word = words[c_idx]
        # 현재 처리할 단어로 현재 처리할 문장이 시작된다면
        if processing_sentence.startswith(word):
            # 해당 문장을 구성하는 단어로 추가하고
            current['words'].append((start, end, word))
            # 해당 단어 부분만큼 삭제해줌
            processing_sentence = processing_sentence[len(word):]
            # 다음 단어를 처리하기 위해 인덱스를 증가시킴
            c_idx += 1
        # 처리해야 할 문자가 남았으나 word로 시작되지 않는 경우, STT 결과와 kiwi 결과가 다른 깨진 문장임
        else:
            # 깨져 있는 문장의 앞 부분을 삭제하고 (`안녕` + `하세요` 중 `안녕`만 현 문장에 포함된 것이므로, `안녕`을 뒤의 문장으로 붙여주는 것)
            current['text'] = current['text'][:-len(word)]
            # 남은 부분을 다음 문장의 앞에 붙여줌
            splitted_sentences[s_idx+1] = processing_sentence + splitted_sentences[s_idx+1]
            # 현재 문장이 깨져 있음을 표시하고, 다음 문장으로 넘어감 (word는 아직 처리되지 않았으므로 c_idx는 그대로)
            broken = True
            break
    
    current['broken'] = broken
    current['start'] = current['words'][0][0]
    current['end'] = current['words'][-1][1]
    reconstructed_segments.append(current)


In [6]:
reconstructed_segments

[{'start': 470,
  'end': 6780,
  'text': '독일 제국의 철혈재상 비스마르크가 독일을 통일하고 보니 세계의 식민지는 이미 열강들의 차지였습니다.',
  'words': [(470, 740, '독일'),
   (770, 1140, '제국의'),
   (1150, 1640, '철혈재상'),
   (1690, 2420, '비스마르크가'),
   (2550, 2940, '독일을'),
   (2950, 3400, '통일하고'),
   (3400, 3660, '보니'),
   (4010, 4400, '세계의'),
   (4400, 4940, '식민지는'),
   (4970, 5320, '이미'),
   (5390, 6040, '열강들의'),
   (6110, 6780, '차지였습니다.')],
  'broken': False},
 {'start': 7550,
  'end': 11920,
  'text': '단독 진출은 무리라고 판단한 비스마르크는 동양에서 협력자를 찾았습니다.',
  'words': [(7550, 7900, '단독'),
   (7930, 8300, '진출은'),
   (8300, 8740, '무리라고'),
   (8740, 9100, '판단한'),
   (9130, 9820, '비스마르크는'),
   (9930, 10500, '동양에서'),
   (10630, 11280, '협력자를'),
   (11330, 11920, '찾았습니다.')],
  'broken': False},
 {'start': 12770,
  'end': 16560,
  'text': '당시 중국의 실력자인 이용장이 적합해 보였습니다.',
  'words': [(12770, 13100, '당시'),
   (13150, 13560, '중국의'),
   (13560, 14140, '실력자인'),
   (14170, 14860, '이용장이'),
   (15130, 15760, '적합해'),
   (15950, 16560, '보였습니다.')],
  'broken':

### 보완
깨진 문장의 끝 부분을 다음 문장의 앞 부분으로 넣어서 정상 처리 해주면, 앞의 깨진 문장이 지나치게 짧아지는 문제가 있음
이는 깨진 문장이 이어지는 문장의 시작 부분이나 kiwi의 오작동으로 인해 서로 다른 두 개의 문장으로 인식되어 발생하는 문제이므로,
뒤의 문장에 앞의 문장을 병합하는 알고리즘을 작성했다.

단, 알고리즘이 좀 불안정한 것 같아서(...) 모듈화 할 때는 따로 해당 로직을 넣지는 않겠다.

In [7]:
# s_idx = 0

# while True:
#     if s_idx >= len(reconstructed_sentences) - 1:
#         break
#     else:
#         sentence = reconstructed_sentences[s_idx]
#         if sentence['broken']:
#             # 연속해서 깨져 있는 문장들을 찾음
#             _s_idx = s_idx
#             while reconstructed_sentences[_s_idx]['broken']:
#                 _s_idx += 1

#             # reconstructed_sentences의 맨 끝 요소까지 깨져 있을 때
#             merge_boundary = _s_idx if _s_idx == len(reconstructed_sentences) else _s_idx + 1
            
#             # 맨 처음의 깨진 문장 뒤로 깨진 문장들을 모두 합침
#             for i in range(s_idx+1, merge_boundary):
#                 sentence['full_text'] += ' ' + reconstructed_sentences[i]['full_text']
#                 sentence['words'] += reconstructed_sentences[i]['words']
            
#             for i in range(s_idx+1, min(len(reconstructed_sentences), merge_boundary)):
#                 reconstructed_sentences.pop(i)
        
#         s_idx += 1


# 메타 정보인 broken을 삭제
for sentence in reconstructed_segments:
    del sentence['broken']

In [8]:
# 결과 출력
reconstructed_segments

[{'start': 470,
  'end': 6780,
  'text': '독일 제국의 철혈재상 비스마르크가 독일을 통일하고 보니 세계의 식민지는 이미 열강들의 차지였습니다.',
  'words': [(470, 740, '독일'),
   (770, 1140, '제국의'),
   (1150, 1640, '철혈재상'),
   (1690, 2420, '비스마르크가'),
   (2550, 2940, '독일을'),
   (2950, 3400, '통일하고'),
   (3400, 3660, '보니'),
   (4010, 4400, '세계의'),
   (4400, 4940, '식민지는'),
   (4970, 5320, '이미'),
   (5390, 6040, '열강들의'),
   (6110, 6780, '차지였습니다.')]},
 {'start': 7550,
  'end': 11920,
  'text': '단독 진출은 무리라고 판단한 비스마르크는 동양에서 협력자를 찾았습니다.',
  'words': [(7550, 7900, '단독'),
   (7930, 8300, '진출은'),
   (8300, 8740, '무리라고'),
   (8740, 9100, '판단한'),
   (9130, 9820, '비스마르크는'),
   (9930, 10500, '동양에서'),
   (10630, 11280, '협력자를'),
   (11330, 11920, '찾았습니다.')]},
 {'start': 12770,
  'end': 16560,
  'text': '당시 중국의 실력자인 이용장이 적합해 보였습니다.',
  'words': [(12770, 13100, '당시'),
   (13150, 13560, '중국의'),
   (13560, 14140, '실력자인'),
   (14170, 14860, '이용장이'),
   (15130, 15760, '적합해'),
   (15950, 16560, '보였습니다.')]},
 {'start': 17190,
  'end': 24220,
  'text': '비스마

### LPM 분석

In [9]:
lines = []

# 문장별 LPM(Letter Per Minute) 계산
for cur in reconstructed_segments:
    sentence = cur['text']
    words = cur['words']

    sentence_start = words[0][0]
    sentence_end = words[-1][1]
    period_min = (sentence_end - sentence_start) / 1000 / 60
    total_words_len = sum(map(len, words))
    lpm = int(total_words_len / period_min)

    if lpm > 500:
        lines.append(f"겁나 빠름 [{lpm}] : {sentence}")
        pass
    elif lpm > 400:
        lines.append(f'빠름 [{lpm}] : {sentence}')
        pass
    elif lpm > 350:
        # lines.append(f'중간 정도 [{lpm}] : {sentence}')
        pass
    else:
        lines.append(f'느림 [{lpm}] : {sentence}')
        pass

# 결과 파일로 작성
with open('./samples/kiwi_splitted_lpm.txt', 'w') as f:
    f.write('\n'.join(lines))
